## A notebook to seamlessly take blast output to GO Slim list

This is a notebook meant to run in a working directory. Please set working directory as variable in next cell

In [1]:
wd="/Users/sr320/Documents/GitHub/nb-2019/P_generosa/wd/022319"


In [2]:
cd $wd

/Users/sr320/Documents/GitHub/nb-2019/P_generosa/wd/022319


### In this directory you will need three files
1) blastout file in format `-6`    
2) Uniprot GO annotation file (340M) available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted`    
3) GOslim file available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted`

In [5]:
#checking if files in directory
!ls 

GO-GOslim.sorted
Pgenerosa_v070.a.makergene.blastx.uniprot.tab
uniprot-SP-GO.sorted


### Set blastout file as variable

In [4]:
blastout="Pgenerosa_v070.a.makergene.blastx.uniprot.tab"


# That should be the last thing you have to Type!

In [5]:
!head -2 $blastout

PGA_scaffold171564__1_contigs__length_6010:2907-4279	sp|Q7Z020|TRPA1_DROME	74.419	43	11	0	1154	1282	912	954	7.23e-11	68.6
PGA_scaffold19103__1_contigs__length_6100:3445-5297	sp|C1DFA8|ALLA_AZOVD	36.364	77	47	1	303	79	70	146	2.50e-09	60.1


In [6]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [7]:
!head -2 _blast-sep.tab

PGA_scaffold171564__1_contigs__length_6010:2907-4279	sp	Q7Z020	TRPA1_DROME	74.419	43	11	0	1154	1282	912	954	7.23e-11	68.6
PGA_scaffold19103__1_contigs__length_6100:3445-5297	sp	C1DFA8	ALLA_AZOVD	36.364	77	47	1	303	79	70	146	2.50e-09	60.1


In [8]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $3, $1, $13}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [9]:
!head -2 _blast-sort.tab

A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	3.33e-09
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	8.97e-09


In [10]:
!head uniprot-SP-GO.sorted

A0A023GPI8	LECA_CANBL	reviewed	Lectin alpha chain (CboL) [Cleaved into: Lectin beta chain; Lectin gamma chain]		Canavalia boliviana	237			mannose binding [GO:0005537]; metal ion binding [GO:0046872]	mannose binding [GO:0005537]; metal ion binding [GO:0046872]	GO:0005537; GO:0046872
A0A023GPJ0	CDII_ENTCC	reviewed	Immunity protein CdiI	cdiI ECL_04450.1	Enterobacter cloacae subsp. cloacae (strain ATCC 13047 / DSM 30054 / NBRC 13535 / NCDC 279-56)	145					
A0A023PXA5	YA19A_YEAST	reviewed	Putative uncharacterized protein YAL019W-A	YAL019W-A	Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)	189					
A0A023PXB0	YA019_YEAST	reviewed	Putative uncharacterized protein YAR019W-A	YAR019W-A	Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)	110					
A0A023PXB5	IRC2_YEAST	reviewed	Putative uncharacterized membrane protein IRC2 (Increased recombination centers protein 2)	IRC2 YDR112W	Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)	102		integ

In [15]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,6 \
> __blast-annot.tab

In [17]:
!head __blast-annot.tab

A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	Autotransporter adhesin UpaG
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	E3 ubiquitin-protein ligase rnf213-beta (EC 2.3.2.27) (EC 3.6.4.-) (Mysterin-B) (Mysterin-beta) (RING finger protein 213-B) (RING finger protein 213-beta) (RING-type E3 ubiquitin transferase rnf213-beta)
A0A0R4I9Y1	PGA_scaffold283089__1_contigs__length_1038:152-480	E3 ubiquitin-protein ligase rnf213-beta (EC 2.3.2.27) (EC 3.6.4.-) (Mysterin-B) (Mysterin-beta) (RING finger protein 213-B) (RING finger protein 213-beta) (RING-type E3 ubiquitin transferase rnf213-beta)
A0A0R4I9Y1	PGA_scaffold283090__1_contigs__length_1068:548-918	E3 ubiquitin-protein ligase rnf213-beta (EC 2.3.2.27) (EC 3.6.4.-) (Mysterin-B) (Mysterin-beta) (RING finger protein 213-B) (RING finger protein 213-beta) (RING-type E3 ubiquitin transferase rnf213-beta)
A0A0R4I9Y1	PGA_scaffold283091__1_contigs__length_1038:558-886	E3 ubiquitin-protein ligase rnf213-beta (EC 2.3.2

## The following is a script modidified from Sam

In [25]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [26]:
!head _blast-GO-unfolded.tab

A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0009279
A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0009405
A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0009986
A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0016021
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0004842
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0005829
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0016887
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0046872
A0A0R4I9Y1	PGA_scaffold283089__1_contigs__length_1038:152-480	GO:0004842
A0A0R4I9Y1	PGA_scaffold283089__1_contigs__length_1038:152-480	GO:0005829


In [38]:
!cat _blast-GO-unfolded.tab | awk -v OFS='\t' '$1=$1' | head

A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0009279
A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0009405
A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0009986
A0A0H2VCA1	PGA_scaffold53906__1_contigs__length_4067:460-2508	GO:0016021
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0004842
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0005829
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0016887
A0A0R4I9Y1	PGA_scaffold283088__1_contigs__length_1068:150-520	GO:0046872
A0A0R4I9Y1	PGA_scaffold283089__1_contigs__length_1038:152-480	GO:0004842
A0A0R4I9Y1	PGA_scaffold283089__1_contigs__length_1038:152-480	GO:0005829
awk: write error on /dev/stdout
 input record number 444, file 
 source line number 1
cat: stdout: Broken pipe


In [35]:
!awk -v OFS='\t' '{print $3, $1, $2}' < _blast-GO-unfolded.tab | sort \
> _blast-GO-unfolded.sorted2

In [36]:
!head _blast-GO-unfolded.sorted2

	A0JNG7	PGA_scaffold12__71_contigs__length_50438331:42801765-42801930
	A0JNG7	PGA_scaffold12__71_contigs__length_50438331:42808273-42830085
	A0JPQ7	PGA_scaffold65427__1_contigs__length_16157:10372-15838
	A2VDD2	PGA_scaffold284723__1_contigs__length_11064:8758-9184
	A2VDD2	PGA_scaffold284726__1_contigs__length_11191:8819-9245
	A2VDD2	PGA_scaffold284737__1_contigs__length_11064:1880-2306
	A2VDD2	PGA_scaffold284740__1_contigs__length_11191:1946-2372
	A2VDD2	PGA_scaffold38337__1_contigs__length_10439:5444-7907
	A2VDD2	PGA_scaffold65668__1_contigs__length_7967:6830-7253
	A2VDD2	PGA_scaffold6__104_contigs__length_61759565:47442001-47442556


In [27]:
!sort -k3 _blast-GO-unfolded.tab > _blast-GO-unfolded.sorted

In [32]:
!head  _blast-GO-unfolded.sorted

A0JNG7	PGA_scaffold12__71_contigs__length_50438331:42801765-42801930
A0JNG7	PGA_scaffold12__71_contigs__length_50438331:42808273-42830085
A0JPQ7	PGA_scaffold65427__1_contigs__length_16157:10372-15838
A2VDD2	PGA_scaffold284723__1_contigs__length_11064:8758-9184
A2VDD2	PGA_scaffold284726__1_contigs__length_11191:8819-9245
A2VDD2	PGA_scaffold284737__1_contigs__length_11064:1880-2306
A2VDD2	PGA_scaffold284740__1_contigs__length_11191:1946-2372
A2VDD2	PGA_scaffold38337__1_contigs__length_10439:5444-7907
A2VDD2	PGA_scaffold65668__1_contigs__length_7967:6830-7253
A2VDD2	PGA_scaffold6__104_contigs__length_61759565:47442001-47442556


In [31]:
!head -1 GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P


In [28]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $3, $1, $5, $6}' \
> Blastquery-GOslim.tab

In [29]:
!head Blastquery-GOslim.tab